In [6]:
%pylab inline
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import pickle

Populating the interactive namespace from numpy and matplotlib


## Get the historical prime rates using scraping

In [199]:
test_df = pd.read_html('https://www.jpmorganchase.com/corporate/About-JPMC/historical-prime-rate.htm')

In [216]:
prime_rate_df = test_df[0].iloc[2:,:]
prime_rate_df.columns = ['date','rate']
prime_rate_df['date'] = pd.to_datetime(df['date'])
prime_rate_df['rate'] = prime_rate_df['rate'].map(lambda x: x[:-1])
prime_rate_df['rate'] = prime_rate_df['rate'].astype('float64')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [218]:
prime_rate_df.to_csv('historic_prime_rates.csv')

## Load vix history

In [267]:
vix1df = pd.read_csv('vixcurrent.csv') # 2003 - present
vix2df = pd.read_excel('vixarchive.xls') #'90s - 2003

In [268]:
vix1df.columns = ['date','vix_open','vix_high','vix_low','vix_close']
vix2df.columns = ['date','vix_open','vix_high','vix_low','vix_close']

In [269]:
#total_vix = pd.concat(vix1df.iloc[1:,:],vix2df.iloc[1:,:])
vix1df = vix1df.iloc[1:,:]
vix2df = vix2df.iloc[1:,:]
frames = [vix1df,vix2df]

In [270]:
total_vix = pd.concat(frames)

In [271]:
for c in total_vix.columns[1:]:
    total_vix[c] = total_vix[c].astype('float64')

In [272]:
total_vix['date'] = pd.to_datetime(total_vix['date'])

In [273]:
total_vix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7317 entries, 1 to 3532
Data columns (total 5 columns):
date         7317 non-null datetime64[ns]
vix_open     6809 non-null float64
vix_high     6809 non-null float64
vix_low      6809 non-null float64
vix_close    7314 non-null float64
dtypes: datetime64[ns](1), float64(4)
memory usage: 343.0 KB


In [275]:
total_vix.to_csv('vix_prices.csv')

## Merge 2 datasets together for regression

In [274]:
#prime_rate_df.set_index(pd.DatetimeIndex(prime_rate_df['date']))
#total_vix.set_index(pd.DatetimeIndex(total_vix['date']))

In [276]:
total_vix.head()

,date,vix_open,vix_high,vix_low,vix_close
1,2004-01-02,17.96,18.68,17.54,18.22
2,2004-01-05,18.45,18.49,17.44,17.49
3,2004-01-06,17.66,17.67,16.19,16.73
4,2004-01-07,16.72,16.75,15.50,15.50
5,2004-01-08,15.42,15.68,15.32,15.61


In [278]:
total_vix.join

(7317, 5)

In [279]:
prime_rate_df.head()

,date,rate
2,1950-09-22,5.0
3,1951-01-08,5.2
4,1951-10-17,5.0
5,1951-12-19,4.7
6,1953-04-27,4.5


## Terrible site not to scrape from

In [14]:
#can I pull the Fed Prime Rates?
url = 'http://www.fedprimerate.com/wall_street_journal_prime_rate_history.htm'
page = requests.get(url)
page.status_code

200

In [168]:
soup=BeautifulSoup(page.content,"html")

In [169]:
rows = soup.find_all(class_='p-1-b-c')

In [170]:
rlist = []
for i in rows[4:520]:
     rlist.append(i.get_text())


In [171]:
date = []
rate = []
for k,v in enumerate(rlist):
    if k % 2 == 0:
        rate.append(v)
    else:
        date.append(v)

    

In [172]:
df_dict = {'date':date,
           'rate':rate}
df = pd.DataFrame.from_dict(df_dict)

In [173]:
df.rate = df.rate.shift(-1)

In [180]:
df.drop(253, inplace=True)
df.reset_index(inplace=True)

In [181]:
df.drop(columns=['index'], inplace=True)

In [182]:
df.date = df.date.str.replace('\n ', '', regex=False)

In [183]:
df.tail()

,date,rate
252,"December 14, 2017",4.50
253,"March 22, 2018",4.75
254,"June 14, 2018",5.00
255,"September 27, 2018",5.25
256,"December 20, 2018",NaN


In [184]:
df['date'] = pd.to_datetime(df['date'])

In [185]:
df.iloc[256][1] = 5.25

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [186]:
df.set_value(256, 'rate', 5.5)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """Entry point for launching an IPython kernel.


,date,rate
0,1947-12-01,1.75
1,1948-08-01,2
2,1950-09-22,2.25
3,1951-01-08,2.5
4,1951-10-17,2.75
5,1951-12-19,3
6,1953-04-27,3.25
7,1954-03-17,3
8,1955-08-04,3.25
9,1955-10-14,3.5


In [187]:
#df['rate'].astype('float64')
print (df['rate'].to_string())

0                                  1.75
1                                     2
2                                  2.25
3                                   2.5
4                                  2.75
5                                     3
6                                  3.25
7                                     3
8                                  3.25
9                                   3.5
10                                 3.75
11                                    4
12                                  4.5
13                                    4
14                                  3.5
15                                    4
16                                  4.5
17                                    5
18                                  4.5
19                                    5
20                                  5.5
21                                 5.75
22                                    6
23                                 5.75
24                                  5.5


In [160]:
df.iloc[100:194,1] = np.nan

In [167]:
df.to_csv('starter.csv')

In [166]:
rlist[199:210]

['January \n            9, 1975',
 '10.25',
 'January \n            15, 1975',
 'January \n            20, 1975',
 'January \n            28, 1975',
 'February \n            3, 1975',
 'February \n            10, 1975',
 'February \n            18, 1975',
 'February \n            24, 1975',
 'March \n            5, 1975',
 'March \n            10, 1975']

In [74]:
d1[2]

'December \n            20, 2018'

In [63]:
d1[0]

'\n- U.S. Prime Rate:\xa0Current \n        Prime Rate \xa0/ Average \n        \xa0/\xa0 Median \n        \xa0/\xa0 Mode \n        \xa0/\xa0 All-Time \n        High -\n\n - Monthly \n        Prime Rate History \xa0\xa0\xa0\xa0 | \xa0\xa0\xa0\xa0\xa0 Prime Rate Chart -\n\nPrime Rate \xa0|\xa0Prime \n        Rate Forecast \xa0|\xa0Certificate \n        of Deposit \xa0|\xa0 Balance \n        Transfer |\xa0 SITEMAP\n\n        \xa0Economy\xa0\xa0| \n        \xa0LIBOR \n        | FREE \n        Credit Reports \xa0|\xa0 Prime \n        Rate FAQ | \xa0Mortgage \n        Rates \n'

In [198]:
rows[205]

<td class="p-1-b-c" width="42%" x:num="27409"><b><font face="Verdana, Arial, Helvetica, sans-serif" size="2">January 
            15, 1975</font></b></td>

In [188]:
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html>
<head>
<link href="pointer-css.css" rel="stylesheet" type="text/css"/>
<title>United States Prime Rate History</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="U.S. Prime Rate us current up-to-date info information WSJ Prime Rate wall street journal history historical index change usa united states California New York" name="keywords"/>
<meta content="Prime Rate History" name="description"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
</head>
<body bgcolor="#FFFFF0">
<a name="top"></a>
<table align="CENTER" border="0" cellpadding="0" cellspacing="0" width="750">
<tr>
<td align="center" colspan="2">
<table border="0" cellpadding="0" cellspacing="0" width="750">
<tr>
<td><img border="0" height="1" src="images/shim.gif" width="491"/></td>
<td><img border="0" height="1" src="images/shim.gif" width="234"/></td>
<td><img border="0" height="1" src="imag

In [11]:
def scrape_prime_rates(url,save_csv=False):
    page = requests.get("http://www.fedprimerate.com/wall_street_journal_prime_rate_history.htm")
    soup=BeautifulSoup(page.content,"html.parser")
    rows = soup.find_all("tr")
    return rows
    #output_columns = ['rank', 'title', 'studio', 'totalGross', 'grossTheaters', 'openingGross',
    #                  'openingTheaters', 'open', 'close']
#     cleaned_rows = []
#     #rows[9].find_all("td")[0].get_text()
#     for j in rows[9:109]:
#         row = []
#         [row.append(i.get_text()) for i in j.find_all("td")]
#         cleaned_rows.append(row)
#     movies2013 = pd.DataFrame(cleaned_rows,columns=output_columns)
#     if save_csv == True:
#         movies2013.to_csv('2013_movies.csv')
#     return movies2013

# def get_movie_links(url, pickle=False):
#     page = requests.get("https://www.boxofficemojo.com/yearly/chart/?yr=2013&p=.htm")
#     soup=BeautifulSoup(page.content,"html.parser")
#     rows = soup.find_all("a")
#     hrefs = []
#     for i in rows[51:349]:
#         hrefs.append(i.get('href'))
#     res = []
#     for i in range(0,len(hrefs),3):
#         res.append(hrefs[i])
#     hrefs_list = []
#     for i in res:
#         hrefs_list.append('https://www.boxofficemojo.com'+i)
#     if pickle == True:
#         pickling_on = open("hrefs_list","wb")
#         pickle.dump(hrefs_list, pickling_on)
#         pickling_on.close()
#     return hrefs_list


# # def data_as_dd(keys,rows,df):
# #     d = defaultdict(list)
# #     for index, row in df.iterrows():
# #         d[row[]]


In [12]:
res = scrape_prime_rates(url)

In [13]:
len(res)

381